In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
from PIL import Image
import sys 
import os
import cv2

## Enhance Images

In [ ]:
# smoothing
in_path = 'coco/building-coco/256/original/images/samples/coco/images/val/'
out_path = 'coco/building-coco/256/original/images/samples/coco/images/val_smoothing/'
for afile in os.listdir(in_path):
    im = cv2.imread(in_path+afile)
    masked = cv2.bilateralFilter(im, 15, 75, 75)
    cv2.imwrite(out_path+afile, masked)

In [ ]:
# edge sharpening
in_path = 'coco/building-coco/256/original/images/samples/coco/images/test/'
out_path = 'coco/building-coco/256/original/images/samples/coco/images/test_sharpen/'
for afile in os.listdir(in_path):
    im = cv2.imread(in_path+afile)
    kernel = np.array([[-1,-1,-1], 
                       [-1,9,-1], 
                       [-1,-1,-1]])
    im = cv2.filter2D(im, -1, kernel)
    cv2.imwrite(out_path+afile, im)


### Mask images

In [ ]:
import os, cv2
import numpy as np
import pandas as pd
import random, tqdm
#import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as album
from torchvision.transforms.functional import adjust_sharpness
from skimage import filters

import segmentation_models_pytorch as smp

from segmentation_models_pytorch import utils
import segmentation_models_pytorch.utils.metrics

In [ ]:
DATA_DIR = 'coco/building-coco/256/original/images/samples/coco/images/'

random_crop_size = 256

x_train_dir = os.path.join(DATA_DIR, 'train')
y_train_dir = os.path.join(DATA_DIR, 'train_labels')

x_valid_dir = os.path.join(DATA_DIR, 'val')
y_valid_dir = os.path.join(DATA_DIR, 'val_labels')

x_test_dir = os.path.join(DATA_DIR, 'test')
y_test_dir = os.path.join(DATA_DIR, 'test_labels')

In [ ]:

# Get class names
class_names = ['background', 'building']
# Get class RGB values
class_rgb_values = [[0, 0, 0], [255, 255, 255]]
select_classes = ['background', 'building']

# Get RGB values of required classes
select_class_indices = [class_names.index(cls.lower()) for cls in select_classes]
select_class_rgb_values =  np.array(class_rgb_values)[select_class_indices]

print('Selected classes and their corresponding RGB values in labels:')
print('Class Names: ', class_names)
print('Class RGB values: ', class_rgb_values)

In [ ]:
# helper function for data visualization
def visualize(**images):
    """
    Plot images in one row
    """
    n_images = len(images)
    plt.figure(figsize=(20,8))
    for idx, (name, image) in enumerate(images.items()):
        plt.subplot(1, n_images, idx + 1)
        plt.xticks([]); 
        plt.yticks([])
        # get title from the parameter names
        plt.title(name.replace('_',' ').title(), fontsize=20)
        plt.imshow(image)
    plt.show()

# Perform one hot encoding on label
def one_hot_encode(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes
    # Arguments
        label: The 2D array segmentation image label
        label_values
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
    
    semantic_map = []
    for colour in label_values:
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)

    return semantic_map
    
# Perform reverse one-hot-encoding on labels / preds
def reverse_one_hot(image):
    """
    Transform a 2D array in one-hot format (depth is num_classes),
    to a 2D array with only 1 channel, where each pixel value is
    the classified class key.
    # Arguments
        image: The one-hot format image 
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of 1, where each pixel value is the classified 
        class key.
    """
    x = np.argmax(image, axis = -1)
    return x

# Perform colour coding on the reverse-one-hot outputs
def colour_code_segmentation(image, label_values):
    """
    Given a 1-channel array of class keys, colour code the segmentation results.
    # Arguments
        image: single channel array where each value represents the class key.
        label_values

    # Returns
        Colour coded image for segmentation visualization
    """
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x


class BuildingsDataset(torch.utils.data.Dataset):

    """Massachusetts Buildings Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_rgb_values (list): RGB values of select classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            class_rgb_values=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        
        
        self.image_paths = [os.path.join(images_dir, image_id) for image_id in sorted(os.listdir(images_dir))]
        self.mask_paths = [os.path.join(masks_dir, image_id) for image_id in sorted(os.listdir(masks_dir))]            

        self.class_rgb_values = class_rgb_values
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read images and masks
        image = cv2.cvtColor(cv2.imread(self.image_paths[i]), cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(cv2.imread(self.mask_paths[i]), cv2.COLOR_BGR2RGB)

        # one-hot-encode the mask
        #mask = one_hot_encode(mask, self.class_rgb_values).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask, self.image_paths[i]
        
    def __len__(self):
        # return length of 
        return len(self.image_paths)

In [ ]:
def get_training_augmentation():
    train_transform = [    
        #album.augmentations.transforms.Sharpen( alpha = (0.2, 0.5),lightness = (0.5, 1.0),always_apply = True,p = 0.5,),
        album.RandomCrop(height=random_crop_size, width=random_crop_size, always_apply=True),
        album.OneOf(
            [
                album.HorizontalFlip(p=1),
                album.VerticalFlip(p=1),
                album.RandomRotate90(p=1),
            ],
            p=0.75,
        ),
    ]
    return album.Compose(train_transform)


def get_validation_augmentation():   
    # Add sufficient padding to ensure image is divisible by 32
    test_transform = [
        #album.PadIfNeeded(min_height=1536, min_width=1536, always_apply=True, border_mode=0),
        album.PadIfNeeded(min_height=15, min_width=15, always_apply=True, border_mode=0),
    ]
    return album.Compose(test_transform)


def to_tensor(x, **kwargs):
    #x = adjust_sharpness(Image.fromarray(np.uint8(x)), 5)
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn=None):
    """Construct preprocessing transform    
    Args:
        preprocessing_fn (callable): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    """   
    _transform = []
    if preprocessing_fn:
        _transform.append(album.Lambda(image=preprocessing_fn))
    _transform.append(album.Lambda(image=to_tensor, mask=to_tensor))
        
    return album.Compose(_transform)

ENCODER = 'resnet50'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = class_names
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

if os.path.exists(os.path.join('./','best_model_128_unet.pth')):
    best_model = torch.load(os.path.join('./','best_model_128_unet.pth'), map_location=DEVICE)
    print('Loaded UNet model.')
else:
    print('fail to load model')



In [ ]:
# create test dataloader (with preprocessing operation: to_tensor(...))
test_dataset = BuildingsDataset(
    os.path.join('coco/building-coco/512','test'), #'../data/tiff_v3/128_modified_pre/'
    os.path.join('coco/building-coco/512','test'),
    # x_test_dir, 
    # y_test_dir, 
    # x_train_dir, 
    # y_train_dir, 
    augmentation=get_validation_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    class_rgb_values=select_class_rgb_values,
)

test_dataloader = DataLoader(test_dataset)

# test dataset for visualization (without preprocessing transformations)
test_dataset_vis = BuildingsDataset(
    os.path.join('coco/building-coco/512','test'),
    os.path.join('coco/building-coco/512','test'),
    # x_test_dir, 
    # y_test_dir, 
    # x_train_dir, 
    # y_train_dir, 
    augmentation=get_validation_augmentation(),
    class_rgb_values=select_class_rgb_values,
)

# get a random test image/mask index
random_idx = random.randint(0, len(test_dataset_vis)-1)
image, mask, img_path = test_dataset_vis[random_idx]

# visualize(
#     original_image = image,
#     ground_truth_mask = colour_code_segmentation(reverse_one_hot(mask), select_class_rgb_values),
#     one_hot_encoded_mask = reverse_one_hot(mask)
# )


In [ ]:
# Center crop padded image / mask to original image dims
def crop_image(image, target_image_dims=[2500,2500,3]):
   
    target_size = target_image_dims[0]
    image_size = len(image)
    padding = (image_size - target_size) // 2

    return image[
        padding:image_size - padding,
        padding:image_size - padding,
        :,
    ]

In [ ]:
sample_preds_folder = os.path.join('coco/building-coco/512/','tmp')
if not os.path.exists(sample_preds_folder):
    os.makedirs(sample_preds_folder)
    

In [ ]:

for idx in range(len(test_dataset)):

    image, gt_mask, img_path = test_dataset[idx]
    image_vis = crop_image(test_dataset_vis[idx][0].astype('uint8'))
    gt_mask_vis = crop_image(test_dataset_vis[idx][1].astype('uint8'))
    x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
    # Predict test image
    pred_mask = best_model(x_tensor)
    pred_mask = pred_mask.detach().squeeze().cpu().numpy()
    # Convert pred_mask from `CHW` format to `HWC` format
    pred_mask = np.transpose(pred_mask,(1,2,0))
    # Get prediction channel corresponding to building
    pred_building_heatmap = pred_mask[:,:,select_classes.index('building')]
    pred_mask = crop_image(colour_code_segmentation(reverse_one_hot(pred_mask), select_class_rgb_values))
    # Convert gt_mask from `CHW` format to `HWC` format
    #gt_mask = np.transpose(gt_mask,(1,2,0))
    #gt_mask = crop_image(colour_code_segmentation(reverse_one_hot(gt_mask), select_class_rgb_values))
    fname = img_path.split('/')[-1]
    nimg = image_vis.copy()
    nimg[pred_building_heatmap<0.02] = 0
    # kernel = np.array([[-1,-1,-1], 
    #                    [-1,9,-1], 
    #                    [-1,-1,-1]])
    # im = gt_mask_vis.copy()
    # im = cv2.filter2D(im, -1, kernel)
    
    # gt_mask_vis[pred_building_heatmap>0.05] = im[pred_building_heatmap>0.05]
#     cv2.imwrite(os.path.join(sample_preds_folder, f"{img_path}.png"), np.hstack([image_vis, gt_mask, pred_mask])[:,:,::-1])
    cv2.imwrite(os.path.join(sample_preds_folder, f"{fname}"), nimg)
    
    
    visualize(
        original_image = image_vis,
        #ground_truth_mask = nimg,
        predicted_mask = pred_mask,
        selected_mask = nimg,
        predicted_building_heatmap = pred_building_heatmap
    )

In [ ]:
import matplotlib.pyplot as plt

plt.hist(pred_building_heatmap, bins=10)

### Test YOLO building model

In [ ]:
import os 

#in_path = 'coco/building-coco/256/original/images/samples/coco/images/val/'
in_path = 'coco/building-coco/512/test_masked/'  #'coco/building-coco/512/test/'
weights = './weights/yolov7-custom-128-512-masked/weights/best.pt' #'./weights/best_256_masked_sharpen.pt'
filelist = sorted(os.listdir(in_path))

for afile in filelist:
    if len(afile.split('.')[0])>1:
        comm = f'python detect.py  --save-txt --save-conf --weights {weights} --conf 0.25  --img-size 512 --source {in_path + afile}'
        print(comm)
        os.system(comm)


In [ ]:
in_path = 'runs/detect/'
labels = []
for afolder in os.listdir(in_path):
    if len(afolder.split('.')[0])>0:
        file = os.listdir(in_path+afolder+'/labels')
        if len(file)>0:
            file = file[0]
            with open(in_path+afolder+'/labels/'+file,'r') as f:
                for aline in f.readlines():
                    labels.append(aline[:-1].split(' '))
            
        else:
            print('no labels')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np_labels = np.array(labels)
plt.hist(np_labels[:,5], bins=10)
np_labels.shape

In [ ]:
# Decoding according to the .yaml file class names order
import os

decoding_of_predictions ={0: 'undamagedcommercialbuilding', \
                          1: 'undamagedresidentialbuilding', \
                          2: 'damagedresidentialbuilding', \
                          3: 'damagedcommercialbuilding', \
                          4: 'background'}

directory = 'challenge_1_submission_images/Validation_Data_JPEG'
directory = 'runs/detect/'
# Directory to store outputs
results_directory = 'results/Validation_Data_Results'

# Create submission directory if it doesn't exist
if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# Loop through each file in the directory
for folder in os.listdir(directory):
    # Check if the current object is a file and ends with .jpeg
    filename = ''
    if len(folder.split('.')[0])>0:
        filename = os.listdir(directory+folder+'/labels')
    print(filename)
    if len(filename)>0:
        filename = filename[0]
        if os.path.isfile(os.path.join(directory+folder+'/labels', filename)) and filename.lower().endswith('.txt'):
            # Perform operations on the file
            file_path = os.path.join(directory+folder+'/labels', filename)
            print(file_path)
            # print("Making a prediction on ", filename)
            # results = model.predict(file_path, save=True, iou=0.5, save_txt=True, conf=0.25)
            
            # for r in results:
            #     conf_list = r.boxes.conf.numpy().tolist()
            #     clss_list = r.boxes.cls.numpy().tolist()
            #     original_list = clss_list
            #     updated_list = []
            #     for element in original_list:
            #          updated_list.append(decoding_of_predictions[int(element)])
    
            # bounding_boxes = r.boxes.xyxy.numpy()
            # confidences = conf_list
            # class_names = updated_list
    
            # # Check if bounding boxes, confidences and class names match
            # if len(bounding_boxes) != len(confidences) or len(bounding_boxes) != len(class_names):
            #     print("Error: Number of bounding boxes, confidences, and class names should be the same.")
            #     continue
            with open(file_path,'r') as f:
                bounding_boxes = f.readlines()
                
                bounding_boxes = [b[:-1].split(' ') for b in bounding_boxes]
            text_file_name = os.path.splitext(filename)[0]
            # Creating a new .txt file for each image in the submission_directory
            with open(os.path.join(results_directory, f"{text_file_name}.txt"), "w") as file:
                for i in range(len(bounding_boxes)):
                    # Get coordinates of each bounding box
                    class_name, confidence, left, top, right, bottom = bounding_boxes[i]
                    # Write content to file in desired format
                    file.write(f"{decoding_of_predictions[int(class_name)]} {confidence} {left} {top} {right} {bottom}\n")
            print("Output files generated successfully.")


In [ ]:
# Define your source directory and the destination where the zip file will be created
source_dir = results_directory
destination_zip = 'submission'

# Create a zip file from the directory
shutil.make_archive(destination_zip, 'zip', source_dir)

print(f"Directory {source_dir} has been successfully zipped into {destination_zip}.")

In [ ]:
# copy masks to folder

list_path = 'coco/building-coco/256/original/samples/YOLODataset/images/train/'
in_path = 'coco/building-coco/256/masks_jpeg/'
out_path = 'coco/building-coco/256/original/samples/YOLODataset/masks/train/'
for aname in os.listdir(list_path):
    
    if aname.replace('.png','.jpg') in os.listdir(in_path):
        
        shutil.copy(in_path+aname.replace('.png','.jpg'), out_path)

In [ ]:
# mask images

list_path = 'coco/building-coco/256/original/samples/YOLODataset/'
for aname in os.listdir(list_path + 'masks/val'):
    img = cv2.imread(list_path + 'images/val/' + aname.replace('.jpg','.png'))
    mask = cv2.imread(list_path + 'masks/val/' + aname)
    
    mask[mask<200] = 0
    mask[mask>100] = 255
    masked = img.copy()
    masked[mask==0] = 0
    cv2.imwrite(list_path + 'masked/val/' + aname,masked)

In [ ]:
# copy labels to folder

list_path = 'coco/building-coco/256/original/samples/YOLODataset/masked/images/val/'
in_path = 'coco/building-coco/256/original/samples/YOLODataset/labels/val/'
out_path = 'coco/building-coco/256/original/samples/YOLODataset/masked/labels/val/'
for aname in os.listdir(list_path):
    shutil.copy(in_path+aname.replace('.jpg','.txt'), out_path)

In [ ]:
# write train.txt, test.txt and val.txt

list_path = 'coco/building-coco/256/original/samples//images/val/'
with open('coco/building-coco/256/original/samples/val.txt','w') as f:
    for aname in os.listdir(list_path):
        f.write(aname+'\n')

In [ ]:
from scipy.io import loadmat

In [ ]:
path =  './coco/plant-data/YelpChi.mat'

yelp = loadmat(path)
net_rur = yelp['net_rur']
net_rtr = yelp['net_rtr']
net_rsr = yelp['net_rsr']
yelp_homo = yelp['homo']


In [ ]:
for item in zip(yelp['homo'].todense(), yelp['label'][0]):
    if item[1] >0:
        print(item[0][item[0]!=0])

In [ ]:
yelp['net_rur'].todense().shape

In [ ]:
yelp['label'][0].shape

In [ ]:
import cv2
from PIL import Image
import pyvips
# from imageio.v2 import imread #as imageio_imread, imsave
# img = imread('coco/plant-coco/image_2016-01-10.tiff')
# print(img.shape)
import tifffile
import numpy as np
import matplotlib.pyplot as plt

img = tifffile.imread('coco/plant-coco/image_2016-01-10.tiff')
img_avg = np.average(np.array(img), axis=2)
print(np.array(img_avg).shape)
plt.imshow(img_avg)

# for i in range(12):
#     plt.imshow(np.array(img)[:,:,i])



# def convert_tiff_to_jpeg_one(in_file):
#     if in_file.endswith('.tiff'):
#         x = pyvips.Image.new_from_file(in_file)
        

# in_file_path = 'coco/plant-coco/image_2016-01-10.tiff'
# convert_tiff_to_jpeg_one(in_file_path)

In [ ]:
def convert_tiff_to_jpeg(input_dir,output_dir):
    # check if output_dir exists, if not create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        # check if file is an image (ends with .tif)
        if filename.endswith('.tif'):
            img = Image.open(os.path.join(input_dir, filename))
        
            # check if image is RGB mode, if not convert it
            if img.mode != 'RGB':
                img = img.convert('RGB')
        
            # create new filename, replace .tif with .jpg
            output_filename = os.path.splitext(filename)[0] + '.jpg'
        
            # save the image in JPEG format
            img.save(os.path.join(output_dir, output_filename), 'JPEG')
    print("Conversion from TIFF to JPEG completed.")

In [ ]:
in_path = 'coco/building-coco/128/test_labels'
out_path = 'coco/building-coco/128/test_labels_jpeg'
convert_tiff_to_jpeg(in_path, out_path)

In [ ]:
# Tile images to make sample sets.

In [ ]:
import cv2
from PIL import Image
from numpy import random
import torch
import matplotlib.pyplot as plt
import os


def plot_one_box(x, img, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image img
    
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    #if label:
    label = 'B '+ str(x[-1])
    tf = max(tl - 1, 1)  # font thickness
    t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
    c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
    cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def plot_one_box(x, img, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image img
    
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    #if label:
    label = 'B '+ str(x[-1])
    tf = max(tl - 1, 1)  # font thickness
    t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
    c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
    cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1]*2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

    
def crop_one_box(x, img):
    # Plots one bounding box on image img
    gn = torch.tensor(img.shape)[[1, 0, 1, 0]]
    x = (x[:4]*gn).view(-1).tolist()
    xy, wh = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    return img[(xy[1]-wh[1]//2):(xy[1]+wh[1]//2),(xy[0]-wh[0]//2):(xy[0]+wh[0]//2)]
    
def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y
    

In [ ]:
# Tile YOLO Exp

path = '/Users/lucy/workspace/development/Chronic Disease/source/imaging/building-data/samples/512_v3_original_0.02/'
image_path = '/Users/lucy/workspace/development/Chronic Disease/source/imaging/building-data/samples/512/test/'

folderlist = os.listdir(path)
count = 0
for folder in folderlist:
    if len(folder.split('.')[0])>0:
        print(folder)
        try:
            filename = os.listdir(path + folder+'/labels')
            if len(filename)>0:
                filename = filename[0]
                print(filename)
                
                if len(filename)>0:
                    if os.path.exists(image_path+'/'+filename.replace('txt','png')):
                        img = cv2.imread(image_path+'/'+filename.replace('txt','png'))
                    elif os.path.exists(image_path+'/'+filename.replace('txt','jpg')):
                        img = cv2.imread(image_path+'/'+filename.replace('txt','jpg'))
                    else:
                        break
                    #print(img.shape)
                    with open(path+folder+'/labels/'+filename,'r') as flabel:
                        labels = flabel.readlines()
                        lcount = 0
                        for alabel in labels:
                            lcount +=1
                            alabel = torch.FloatTensor([float(a) for a in alabel[:-1].split(' ')])
                            
                            atile = crop_one_box(alabel[2:6], img)

                            if atile.shape[0]>0 and atile.shape[1]>0:
                                t_path = path+folder+'/'+filename.replace('.txt','')+'_'+str(lcount)+'.png'
                                print(t_path)
                                cv2.imwrite(t_path,atile)
                                # t_path = path+folder+'/'+filename.replace('.txt','')+'_boxed'+str(lcount)+'.png'
                                # #print(t_path)
                                # cv2.imwrite(t_path,aboxed)
            
        except:
            print(path + folder+'/labels')

In [ ]:
import cv2
from PIL import Image
from numpy import random
import torch
import matplotlib.pyplot as plt


def plot_one_box(x, img, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image img
    
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    #if label:
    label = 'B '+ str(x[-1])
    tf = max(tl - 1, 1)  # font thickness
    t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
    c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
    cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def plot_one_box(x, img, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image img
    
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    #if label:
    label = 'B '+ str(x[-1])
    tf = max(tl - 1, 1)  # font thickness
    t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
    c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
    cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1]*2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

    
def crop_one_box(x, img, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image img
    gn = torch.tensor(img.shape)[[1, 0, 1, 0]]
    x = (x[:4]*gn).view(-1).tolist()
    xy, wh = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    return img[(xy[1]-wh[1]//2):(xy[1]+wh[1]//2),(xy[0]-wh[0]//2):(xy[0]+wh[0]//2)]
    
def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y
    
      
path =        '/Users/lucy/workspace/development/Chronic Disease/source/imaging/building-data/samples/512_v3_original_0.02/exp/' 
aimage = cv2.imread(path + 'Validation_Post_Event_001.jpg')
gn = torch.tensor(aimage.shape)[[1, 0, 1, 0]]

tl = round(0.002 * (aimage.shape[0] + aimage.shape[1]) / 2) + 1
tf = max(tl-1, 1)

color = [random.randint(0, 255) for _ in range(3)]

xx = torch.Tensor([0.25, 0.547852, 0.0585938, 0.0566406,0.5])
xyxy = (xywh2xyxy(xx[:-1].view(1,-1))*gn).view(-1).tolist()
xyxy.append(0.5)



aimage_copy = aimage.copy()
newim = crop_one_box(xx[:4], aimage_copy)

#plot_one_box(xyxy, aimage) 

newim_labeled = crop_one_box(xx[:4], aimage)



cv2.imwrite('./inference/images/test.jpg', aimage)
plt.imshow(newim)
plt.show()
plt.imshow(newim_labeled)
plt.show()
plt.imshow(aimage)
plt.show()

In [ ]:
import cv2
from PIL import Image
from numpy import random
import torch
import matplotlib.pyplot as plt


def plot_one_box(x, img, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image img
    
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    #if label:
    label = 'B '+ str(x[-1])
    tf = max(tl - 1, 1)  # font thickness
    t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
    c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
    cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def plot_one_box(x, img, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image img
    
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    #if label:
    label = 'B '+ str(x[-1])
    tf = max(tl - 1, 1)  # font thickness
    t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
    c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
    cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1]*2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

    
def crop_one_box(x, img, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image img
    gn = torch.tensor(img.shape)[[1, 0, 1, 0]]
    x = (x[:4]*gn).view(-1).tolist()
    xy, wh = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    return img[(xy[1]-wh[1]//2):(xy[1]+wh[1]//2),(xy[0]-wh[0]//2):(xy[0]+wh[0]//2)]
    
def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y
    
      
        
aimage = cv2.imread('inference/images/bus.jpg')
gn = torch.tensor(aimage.shape)[[1, 0, 1, 0]]

tl = round(0.002 * (aimage.shape[0] + aimage.shape[1]) / 2) + 1
tf = max(tl-1, 1)

color = [random.randint(0, 255) for _ in range(3)]

xx = torch.Tensor([0.182716, 0.603241, 0.246914, 0.467593, 0.5])
xyxy = (xywh2xyxy(xx[:-1].view(1,-1))*gn).view(-1).tolist()
xyxy.append(0.5)

aimage_copy = aimage.copy()
newim = crop_one_box(xx[:4], aimage_copy)

plot_one_box(xyxy, aimage) 

newim_labeled = crop_one_box(xx[:4], aimage)



cv2.imwrite('./inference/images/test.jpg', aimage)
plt.imshow(newim)
plt.show()
plt.imshow(newim_labeled)
plt.show()
plt.imshow(aimage)
plt.show()

In [ ]:
# python train.py --workers 8 --device 0 --batch-size 4 --data data/building-256-coco.yaml --img 256 256 --cfg cfg/training/yolov7.yaml --name yolov7 --hyp data/hyp.scratch.custom.yaml


# python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source yourvideo.mp4
comm = 'python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --save-txt --save-conf --source ./inference/images/bus.jpg'

os.system(comm)

In [ ]:
# Image Padding

In [ ]:
from PIL import Image, ImageOps
import numpy as np
import cv2

def padding(img, expected_size):
    desired_size = expected_size
    delta_width = desired_size[0] - img.size[0]
    delta_height = desired_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)

In [ ]:
folders = ['t_comm','t_dcomm']
in_path = '../../building-data/samples/training_v18/'
out_path = '../../building-data/samples/training_v18/resized/'
for afolder in folders:
        
    for afile in os.listdir(in_path+afolder):
        if len(afile.split('.')[0])>0:
            img = cv2.imread(in_path+afolder+'/'+afile)
            kernel = np.array([[-1,-1,-1], 
                       [-1, 9,-1],
                       [-1,-1,-1]])
            img = cv2.filter2D(img, -1, kernel)
            img = padding(Image.fromarray(img), [256,256])
            img.save(out_path+afolder+'/'+afile)

In [ ]:
import shutil
import os

in_path = '../../building-data/samples/512_v2_sharpen_0.02/'
#out_path = '../../building-data/samples/v2_128post/resized/res_dmg/'
for i in range(1,10):
    var = '0'+str(i)
    for foldername in os.listdir(in_path+var):
        if len(foldername.split('.')[0])>0:
            folderlist = os.listdir(in_path+var+'/'+foldername)
            for afile in folderlist:
                shutil.copy(in_path+var+'/'+foldername+'/'+afile, in_path+foldername)

In [ ]:
# image sharpening
import cv2
import numpy as np

image = cv2.imread('inference/images/bus.jpg')
kernel = np.array([[-1,-1,-1], 
                   [-1, 9,-1],
                   [-1,-1,-1]])
sharpened = cv2.filter2D(image, -1, kernel) # applying the sharpening kernel to the input image & displaying it.
cv2.imshow('Image Sharpening', sharpened)
cv2.waitKey(0)
cv2.destroyWindow('Image Sharpening')

In [ ]:
# image masking
from PIL import Image
import cv2
import numpy as np

image = cv2.imread('../../building-data/128/pre/selected_jpeg_backup/tile_0_357.jpg')
mask = cv2.imread('../../building-data/128/building/selected_backup_jpeg/tile_0_357.jpg')


In [ ]:
mask[mask<200]=0
mask[mask>100]=255
mask.shape

In [ ]:
mask = np.average(mask, axis=-1)
mask = mask.astype(np.float32)
mask = cv2.merge([mask, mask, mask])
mask.shape

In [ ]:

cv2.imshow("Rectangular Mask", mask)
cv2.waitKey(0)
cv2.destroyWindow('Rectangular Mask')
cv2.waitKey(1)

In [ ]:

masked = image*mask*255 #np.clip(image+mask, 0, 1) #
masked = cv2.bilateralFilter(masked, 15, 75, 75) 
nimage = image.copy()
nimage[mask!=0] = masked[mask!=0]
masked[mask!=0] = 255

In [ ]:

cv2.imshow("Rectangular Mask", mask)
cv2.imshow("Mask Applied to Image", masked)
cv2.imshow("Masked Image", nimage)
cv2.imshow('Original Image', image)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# image masking
import cv2
import numpy as np

image = cv2.imread('inference/images/bus.jpg')

mask = np.zeros(image.shape[:2], dtype="uint8")
cv2.rectangle(mask, (0, 90), (290, 450), 255, -1)
cv2.imshow("Rectangular Mask", mask)


masked = cv2.bitwise_and(image, image, mask=mask)
cv2.imshow("Mask Applied to Image", new_image)

cv2.waitKey(0)
cv2.destroyWindow("Mask Applied to Image")
cv2.destroyWindow('Rectangular Mask')

In [ ]:
# apply mask to region
mask = np.ones([5,5])
region = np.ones([5,5])
region[1:3, 2:3] = 0
mask[1:3,2:3 ] = region[1:3, 2:3]
mask, region

In [ ]:
# Performing averaging blurring on our Cybertruck image
# Filters - left (3,3), middle(5,5), right(9,9)
blurred_1 = np.hstack([
  cv2.blur(img1,(3,3)),
  cv2.blur(img1,(5,5)),
  cv2.blur(img1,(9,9))])  
cv2_imshow(blurred_1)

# Performing averaging blurring on our Logo image
# Filters - left (3,3), middle(5,5), right(9,9)
blurred_2 = np.hstack([
  cv2.blur(img2,(3,3)),
  cv2.blur(img2,(5,5)),
  cv2.blur(img2,(9,9))])  
cv2_imshow(blurred_2)

# Performing Gaussian blurring on our Cybertruck image
Gaussian_blurred_1 = np.hstack([
  cv2.GaussianBlur(img1,(3,3),0),
  cv2.GaussianBlur(img1,(5,5),0),
  cv2.GaussianBlur(img1,(9,9),0)])  
cv2_imshow(Gaussian_blurred_1)

# Performing Gausssian blurring on our Logo image
Gaussian_blurred_2 = np.hstack([
  cv2.GaussianBlur(img2,(3,3),0),
  cv2.GaussianBlur(img2,(5,5),0),
  cv2.GaussianBlur(img2,(9,9),0)])  
cv2_imshow(Gaussian_blurred_2)



In [ ]:
import os, cv2
import numpy as np
import pandas as pd
import random, tqdm
#import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as album
from torchvision.transforms.functional import adjust_sharpness
from skimage import filters


In [ ]:
import segmentation_models_pytorch as smp

from segmentation_models_pytorch import utils
import segmentation_models_pytorch.utils.metrics

In [ ]:
in_path = 'coco/building-coco/128/test/'
out_path = 'coco/building-coco/'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model = torch.load(os.path.join('./','best_model_256.pth'), map_location=DEVICE)

for aimage in os.listdir(in_path):

    image = cv2.imread(in_path+aimage)
    
    x_tensor = torch.FloatTensor(np.transpose(image,(2, 0, 1))).unsqueeze(0)
    
    # Predict test image
    pred_mask = best_model(x_tensor).squeeze().detach().numpy()

    # Convert pred_mask from `CHW` format to `HWC` format
    pred_mask = np.transpose(pred_mask,(1,2,0))
    pred_mask = np.round(np.average(pred_mask, axis=-1), decimals =0)
    pred_mask = cv2.merge([pred_mask, pred_mask, pred_mask])
    nimage = image.copy()
    nimage[pred_mask==0] = pred_mask[pred_mask==0]
    cv2.imshow('original image',image)
    cv2.imshow('mask',pred_mask)
    cv2.imshow('masked image',nimage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    break
    
    
    cv2.imwrite(os.path.join(out_path+'masked', aimage.replace('.tif','.png')), pred_mask)
    cv2.imwrite(os.path.join(out_path+'images', aimage.replace('.tif','.png')), image)
    


In [ ]:
import shutil

in_path = 'coco/building-coco/classification/all_valid/'
out_path = 'coco/building-coco/classification/'
folderlist = os.listdir(in_path)
for afolder in folderlist: # folder 001 - 012
    cfolderlist = os.listdir(in_path+afolder)
    cfolderlist = [f for f in cfolderlist if os.path.isdir(f)]
    for cfolder in cfolderlist: # class folders
        afilelist = os.listdir(in_path+afolder+'/'+cfolder)
        afilelist = [f for f in afilelist if f.split('.')[-1]=='png']
        for afile in afilelist: # images
            shutil.copy(in_path+afolder+'/'+cfolder+'/'+afile, out_path+cfolder)

In [ ]:
# Decoding according to the .yaml file class names order
decoding_of_predictions ={0: 'undamagedcommercialbuilding', 1: 'undamagedresidentialbuilding', 2: 'damagedresidentialbuilding', 3: 'damagedcommercialbuilding'}
#decoding_of_predictions ={3: 'undamagedcommercialbuilding', 2: 'undamagedresidentialbuilding', 1: 'damagedresidentialbuilding', 0: 'damagedcommercialbuilding'}

directory = 'results/challenge_1_submission_images/Validation_Data_JPEG/'
# Directory to store outputs
results_directory = 'results/Validation_Data_Results/'

# Create submission directory if it doesn't exist
if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# Loop through each file in the directory
for afolder in os.listdir(directory):
    # Check if the current object is a file and ends with .jpeg
    if len(afolder.split('.')[0])>0:
        labels = os.listdir(directory+afolder+'/labels/')
        if len(labels)>0:
            filename = labels[0]
            with open(os.path.join(directory+afolder+'/labels', filename), 'r') as f:
                
                text_file_name = os.path.splitext(filename)[0]
                # Creating a new .txt file for each image in the submission_directory
                with open(os.path.join(results_directory, f"{text_file_name}.txt"), "w") as file:
                    
                    for aline in f.readlines():
                        alabel = aline[:-1].split(' ')
                        
                        bounding_boxes = alabel[2:]
                        confidence = alabel[1]
                        class_name = int(alabel[0])
                        # Get coordinates of each bounding box
                        left, top, right, bottom = bounding_boxes
                        # Write content to file in desired format
                        file.write(f"{decoding_of_predictions[class_name]} {confidence} {left} {top} {right} {bottom}\n")
                    print("Output files generated successfully.")